# Basic 2D Trajectories

This notebook demonstrates the generation, summarization, and visualization of basic 2D k-space trajectories using the `trajgen` library.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from trajgen import KSpaceTrajectoryGenerator, Trajectory

# Ensure plots appear inline in the notebook
%matplotlib inline

## 1. 2D Spiral Trajectory

In [ ]:
# Instantiate generator for a 2D spiral
gen_spiral = KSpaceTrajectoryGenerator(
    traj_type='spiral',
    dim=2,
    fov=0.256,            # Field of View in meters
    resolution=0.004,     # Resolution in meters
    n_interleaves=1,      # Single interleaf for a simple demo
    turns=8               # Number of turns in the spiral
)

# Generate trajectory waveforms
kx_spiral, ky_spiral, gx_spiral, gy_spiral, t_spiral = gen_spiral.generate()

# kx, ky are typically (n_interleaves, n_samples_per_interleaf)
# For a single interleaf demo, we take the first one: kx_spiral[0], ky_spiral[0]
kspace_spiral_2d = np.stack([kx_spiral[0], ky_spiral[0]])
gradients_spiral_2d = np.stack([gx_spiral[0], gy_spiral[0]])

# Create Trajectory object
traj_spiral = Trajectory(
    name='2D Spiral Example',
    kspace_points_rad_per_m=kspace_spiral_2d,
    gradient_waveforms_Tm=gradients_spiral_2d, # Optional, can be computed by Trajectory class
    dt_seconds=gen_spiral.dt,
    metadata={'gamma_Hz_per_T': gen_spiral.gamma, 'generator_params': gen_spiral.__dict__}
)

# Display trajectory summary
traj_spiral.summary()

In [ ]:
# Plot the 2D Spiral k-space trajectory
plt.figure(figsize=(6, 6))
plt.plot(kspace_spiral_2d[0, :], kspace_spiral_2d[1, :])
plt.title('2D Spiral K-Space Trajectory')
plt.xlabel('Kx (rad/m)')
plt.ylabel('Ky (rad/m)')
plt.axis('equal')
plt.grid(True)
plt.show()

## 2. 2D Radial Trajectory

In [ ]:
# Instantiate generator for a 2D radial trajectory
gen_radial = KSpaceTrajectoryGenerator(
    traj_type='radial',
    dim=2,
    fov=0.256,
    resolution=0.004,
    n_interleaves=16,  # Number of radial spokes
    use_golden_angle=True # Use golden angle for spoke distribution
)

# Generate trajectory waveforms
kx_radial, ky_radial, gx_radial, gy_radial, t_radial = gen_radial.generate()

# Combine all interleaves (spokes) for the Trajectory object
# kx_radial.ravel() flattens the (n_interleaves, n_samples) array into a single dimension
kspace_radial_2d = np.stack([kx_radial.ravel(), ky_radial.ravel()])
gradients_radial_2d = np.stack([gx_radial.ravel(), gy_radial.ravel()])

# Create Trajectory object
traj_radial = Trajectory(
    name='2D Radial Example',
    kspace_points_rad_per_m=kspace_radial_2d,
    gradient_waveforms_Tm=gradients_radial_2d,
    dt_seconds=gen_radial.dt,
    metadata={'gamma_Hz_per_T': gen_radial.gamma, 'generator_params': gen_radial.__dict__}
)

# Display trajectory summary
traj_radial.summary()

In [ ]:
# Plot the 2D Radial k-space trajectory
plt.figure(figsize=(6, 6))
plt.plot(kspace_radial_2d[0, :], kspace_radial_2d[1, :], '.') # Plot points for clarity
plt.title('2D Radial K-Space Trajectory (16 spokes)')
plt.xlabel('Kx (rad/m)')
plt.ylabel('Ky (rad/m)')
plt.axis('equal')
plt.grid(True)
plt.show()

## 3. 2D EPI Trajectory

In [ ]:
# Instantiate generator for a 2D EPI trajectory
gen_epi = KSpaceTrajectoryGenerator(
    traj_type='epi',
    dim=2,
    fov=0.256,
    resolution=0.008, # Lower resolution for fewer PE lines for clarity
    n_interleaves=32, # Number of phase-encode lines in Ky
)

# Generate trajectory waveforms
kx_epi, ky_epi, gx_epi, gy_epi, t_epi = gen_epi.generate()

# Combine all interleaves (phase-encode lines)
kspace_epi_2d = np.stack([kx_epi.ravel(), ky_epi.ravel()])
gradients_epi_2d = np.stack([gx_epi.ravel(), gy_epi.ravel()])

# Create Trajectory object
traj_epi = Trajectory(
    name='2D EPI Example',
    kspace_points_rad_per_m=kspace_epi_2d,
    gradient_waveforms_Tm=gradients_epi_2d,
    dt_seconds=gen_epi.dt,
    metadata={'gamma_Hz_per_T': gen_epi.gamma, 'generator_params': gen_epi.__dict__}
)

# Display trajectory summary
traj_epi.summary()

In [ ]:
# Plot the 2D EPI k-space trajectory
plt.figure(figsize=(8, 6))
# Plotting every few points can make the raster pattern clearer for dense EPIs
plt.plot(kspace_epi_2d[0, ::10], kspace_epi_2d[1, ::10], '-') 
plt.scatter(kspace_epi_2d[0, 0], kspace_epi_2d[1, 0], color='g', s=100, label='Start') # Mark start
plt.scatter(kspace_epi_2d[0, -1], kspace_epi_2d[1, -1], color='r', s=100, label='End') # Mark end
plt.title('2D EPI K-Space Trajectory (32 PE lines)')
plt.xlabel('Kx (rad/m)')
plt.ylabel('Ky (rad/m)')
plt.axis('equal')
plt.grid(True)
plt.legend()
plt.show()

This concludes the basic 2D trajectory examples. The `Trajectory` object's `summary()` method provides key metrics, and k-space plots allow for visual inspection.